In [1]:
from pathlib import Path

from utils.util import *
%matplotlib inline

discriminator_folder = "../experiment_results/discriminator_0.8_test"
csv_name = "discriminator_pred_on_test.csv"
output_dir = "classifier_analysis"

Path(output_dir).mkdir(parents=True, exist_ok=True)

KeyboardInterrupt: 

In [ ]:
experiments = [ "categorical_crossentropy", "polyloss_ce_01"]
experiments_rename = [ "categorical_crossentropy", "Polyloss_CE"]

In [ ]:

import matplotlib.pyplot as plt


def plot_metrics(experiments):
    # Set up figure and axes
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(40, 20))

    # Plot accuracy and precision for each algorithm
    for i, experiments_name in enumerate(experiments):
        df = pd.read_csv(f"../{os.path.join(DEFAULT_RESULTS_FOLDER, experiments_name, DISCRIMINATOR_METRICS_FILE)}")
        Q1 = df.quantile(0.05)
        Q3 = df.quantile(0.95)
        IQR = Q3 - Q1

        df = df[~((df < (Q1 -  IQR)) | (df > (Q3 +  IQR))).any(axis=1)]
        x = df["epoch"]

        # Compute cumulative average every 3 epochs
        df["Accuracy_cumavg"] = df["Accuracy"].rolling(3).mean()
        df["F1_score_cumavg"] = df["F1-score"].rolling(3).mean()

        # Get the color of the line used for the experiment
        color = ax[0]._get_lines.get_next_color()

        # Add regression line for accuracy
        # slope_acc, intercept_acc = np.polyfit(x, df["Accuracy"], 1)
        ax[0].plot(x, df["Accuracy_cumavg"], label=f"{experiments_rename[i]}", color=color, linestyle='--')
        coefficients = np.polyfit(x, df["Accuracy"], 4)
        p = np.poly1d(coefficients)
        ax[0].plot(x, p(x), color=color, linewidth=4)

        # Add regression line for precision
        ax[1].plot(x, df["F1_score_cumavg"], label=f"{experiments_rename[i]}", color=color, linestyle='--')
        coefficients = np.polyfit(x, df["F1-score"], 4)
        p = np.poly1d(coefficients)
        ax[1].plot(x, p(x), color=color, linewidth=4)

        ax[0].legend(fontsize=14)
        ax[0].set_xlabel("Epoch", fontsize=20)
        ax[0].set_ylabel("Accuracy", fontsize=20)

        ax[1].legend(fontsize=14)
        ax[1].set_xlabel("Epoch", fontsize=20)
        ax[1].set_ylabel("F1-score", fontsize=20)

    ax[0].set_facecolor('black')
    ax[1].set_facecolor('black')
    plt.savefig(os.path.join(output_dir, "discriminator_class_metrics.jpg"))
    plt.show()


In [ ]:
plot_metrics(experiments)

In [ ]:
plot_classifications_confusion_matrix(['../experiment_results/categorical_crossentropy/discriminator_pred_on_test.csv','../GS-AC-GAN/experiment_results/polyloss_ce_01/discriminator_pred_on_test.csv'], output_dir, experiments_rename)

In [ ]:
def get_last_metrics(experiments):
    last_metrics_dfs = []
    for i, experiments_name in enumerate(experiments):
        df = pd.read_csv(f"../{os.path.join(DEFAULT_RESULTS_FOLDER, experiments_name, DISCRIMINATOR_METRICS_FILE)}")
        df = df.tail(1)
        df = pd.DataFrame(df.mean()).T
        df["experiments_name"] = experiments_rename[i]
        last_metrics_dfs.append(df)
    catboost_results = pd.read_csv(os.path.join(output_dir, "catboost_classification_results.csv"))
    catboost_results["experiments_name"] = "CatBoostClassifier"
    last_metrics_dfs.append(catboost_results.tail(1))
    return pd.concat(last_metrics_dfs, ignore_index=True)

all_experiments_scores = get_last_metrics(experiments)
all_experiments_scores.set_index('experiments_name', inplace=True)
all_experiments_scores.drop(["epoch", "Jaccard Similarity Coefficient", "Recall", "ROC AUC", "Precision"], inplace=True, axis=1)
all_experiments_scores = all_experiments_scores.round(4)

df_transposed = all_experiments_scores.transpose()

# plot the bar chart
ax = df_transposed.plot(kind='bar', figsize=(28,8), width=0.9,  fontsize=16)

# set labels
ax.set_title('Performance of experiments - average recent epochs',  fontsize=22)
ax.set_xlabel('Score Type')
ax.set_ylabel('Score')

# show the legend
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

# add numbers to bars
for i, rect in enumerate(ax.containers):
    ax.bar_label(rect, labels=all_experiments_scores.iloc[i].astype(str), fontsize=16)

plt.savefig(os.path.join(output_dir,"discriminator_compare_catboost"))
plt.show()